# Concurrent Playing Cards
Channel과 MutableStateFlow를 활용해 덱의 기능을 구현하면
동시에 여러 플레이어가 정신없이 카드를 하나 가져와 하나 버리는 것을 시뮬레이션할 수 있다!

In [1]:
enum class Symbol { SPADE, HEART, DIA, CLUB; } // 모양

enum class Card(val face: String, val back: String = "🂠") { // 카드의 종류
    SA("🂡"),S2("🂢"),S3("🂣"),S4("🂤"),S5("🂥"),S6("🂦"),
    S7("🂧"),S8("🂨"),S9("🂩"),S0("🂪"),SJ("🂫"),SQ("🂭"),SK("🂮"),
    HA("🂱"),H2("🂲"),H3("🂳"),H4("🂴"),H5("🂵"),H6("🂶"),
    H7("🂷"),H8("🂸"),H9("🂹"),H0("🂺"),HJ("🂻"),HQ("🂽"),HK("🂾"),
    DA("🃁"),D2("🃂"),D3("🃃"),D4("🃄"),D5("🃅"),D6("🃆"),
    D7("🃇"),D8("🃈"),D9("🃉"),D0("🃊"),DJ("🃋"),DQ("🃍"),DK("🃎"),
    CA("🃑"),C2("🃒"),C3("🃓"),C4("🃔"),C5("🃕"),C6("🃖"),
    C7("🃗"),C8("🃘"),C9("🃙"),C0("🃚"),CJ("🃛"),CQ("🃝"),CK("🃞");
    
    fun symbol() = when(this) {
        in SA..SK -> Symbol.SPADE
        in HA..HK -> Symbol.HEART
        in DA..DK -> Symbol.DIA
        else      -> Symbol.CLUB // 나머지 경우는 
    }
    
    fun color() = when(symbol()) {
        Symbol.SPADE -> "black"
        Symbol.HEART -> "red"
        Symbol.DIA   -> "red"
        Symbol.CLUB  -> "black"
    }
    
    fun rank() = this.ordinal % 13 + 1 // A가 1, 숫자는 숫자값, J,Q,K는 11,12,13
}

In [2]:
data class CardItem(val card: Card, var up: Boolean = false) { // 카드 한장
    fun symbol() = card.symbol()
    fun color() = card.color()
    fun rank() = card.rank()
    fun show() = if(up) card.face else card.back
}

In [3]:
object MyUtil {
    fun readLn(): String {
        try {
            return readln()
        } catch (e: Exception) {
            val msg = e.message
            if (msg != null) {
                val curr = msg.split('\n').last()
                val prefix = "Current input: {\"status\":\"ok\",\"value\":"
                if ( curr.startsWith(prefix) ) {
                    return curr.drop( prefix.length + 1 ).dropLast(1 + 1)
                }
            }
            throw e
        }
    }
    fun readNonNegativeIntUntilSuccess(): Int {
        do {
             val z = this.readLn().toIntOrNull()
             if (z != null && z >= 0) return z
        } while (true)
    }
    val style = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+Symbols+2&display=swap');
.card {
    display: inline-block; 
    font-size: 60pt; height: 62pt;
    font-family: "Noto Sans Symbols 2", sans-serif;
    background: rgba(255,255,255, 1) !important;
}
.deck { margin-right:-32pt; }
</style>
"""
}

In [4]:
HTML(
    Card.values().toList().shuffled()
        .map { CardItem(it,true) }
        .fold(MyUtil.style) { acc, c -> acc +
"""<span class='card deck'
        style='color: ${if (c.up) c.color() else "black"};'>${c.show()}</span>"""
        }
)

🂲 🃔 🂹 🂺 🃗 🂪 🃅 🂫 🂳 🃓 🂤 🃊 🂸 🂷 🃎 🃄 🃛 🃙 🃋 🂶 🂦 🃉 🃆 🂽 🂻 🃘 🃝 🃕 🂭 🃂 🃚 🂥 🃖 🂵 🃒 🂣 🂩 🂧 🂾 🃁 🂮 🂴 🃑 🂱 🃞 🃇 🃍 🃃 🃈 🂢 🂨 🂡

In [5]:
interface Deck {
    fun size(): Int
    fun draw(): CardItem // 덱에서 카드를 뽑기
    fun put(c: CardItem) // 덱에 카드 c를 놓기
}

interface Player {
    val hand: MutableList<CardItem> // 이것도 가능!!
    fun draw(d: Deck) // 덱 d로부터 카드 하나 뽑기
    fun put(d: Deck)  // 카드 하나를 덱 d에 놓기
}

interface ToHTML {
    fun toHTML(): String // 해당 객체를 표현하는 HTML 소스코드 문자열 생성
}

In [6]:
class GameDeck(val items: ArrayDeque<CardItem>) : Deck, ToHTML {
    override public fun size() = items.size
    override public fun draw() = items.removeLast()
    override public fun put(c: CardItem) = items.addFirst(c).also { c.up = false }
    override public fun toHTML() = items.fold(MyUtil.style) { acc, c -> acc +
"""<span class='card deck'
        style='color: ${if (c.up) c.color() else "black"};'>${c.show()}</span>"""
        }
}

In [7]:
val d = GameDeck( ArrayDeque( listOf(Card.SA, Card.S2).map { CardItem(it) } ) )

In [8]:
d.size()

2

In [9]:
HTML(d.toHTML())

🂠 🂠

In [10]:
d.put( CardItem(Card.S3) )
println(d.size())
HTML(d.toHTML())

3


🂠 🂠 🂠

In [11]:
val c2 = d.draw()
HTML(d.toHTML())

🂠 🂠

In [12]:
c2

CardItem(card=S2, up=false)

In [13]:
class ComputerPlayer : Player, ToHTML {
    override val hand: MutableList<CardItem> = mutableListOf()
    override fun draw(d: Deck) {
        val c = d.draw()
        c.up = false
        hand.add( c )
    }
    override fun put(d: Deck) = d.put( hand.removeAt(0) )
    override public fun toHTML() = hand.fold(MyUtil.style) { acc, c -> acc +
"""<span class='card'
        style='color: ${if (c.up) c.color() else "black"};'>${c.show()}</span>"""
        }
}

In [14]:
val cp = ComputerPlayer()
cp.draw(d)

In [15]:
HTML( d.toHTML() )

🂠

In [16]:
HTML( cp.toHTML() )

🂠

In [17]:
cp.draw(d)

In [18]:
HTML( d.toHTML() )

In [19]:
HTML( cp.toHTML() )

🂠 🂠

In [20]:
class HumanPlayer : Player, ToHTML {
    override val hand: MutableList<CardItem> = mutableListOf()
    override fun draw(d: Deck) {
        val c = d.draw()
        c.up = true
        hand.add( c )
    }
    override fun put(d: Deck)  {
        val i = MyUtil.readNonNegativeIntUntilSuccess()
        if (i in 0 .. hand.size-1) {
            d.put( hand.removeAt(i) )
        } else {
            println("잘못된 범위를 입력했으므로 첫번째 카드(index 0)를 버리겠습니다!")
            d.put( hand.removeAt(0) )
        }
    }
    override public fun toHTML() = hand.fold(MyUtil.style) { acc, c -> acc +
"""<span class='card'
        style='color: ${if (c.up) c.color() else "black"};'>${c.show()}</span>"""
        }
}

In [21]:
val d1 = GameDeck( ArrayDeque( listOf(Card.SA, Card.S2).map { CardItem(it) } ) )

In [22]:
HTML( d1.toHTML() )

🂠 🂠

In [23]:
val hp = HumanPlayer()

In [24]:
hp.draw(d1)

In [25]:
HTML( d1.toHTML() )

🂠

In [26]:
HTML( hp.toHTML() )

🂢

In [27]:
hp.draw(d1)

In [28]:
HTML( d1.toHTML() )

In [29]:
HTML( hp.toHTML() )

🂢 🂡

In [30]:
hp.put(d1) // hp가 들고 있는 카드가 두 장이므로 0이나 1을 입력하라

stdin: 0


In [31]:
HTML( d1.toHTML() )

🂠

In [32]:
HTML( hp.toHTML() )

🂡

여기까지는 지난 과제 해설

----

기말과제 관련 내용은 여기서부터 ...

## 기말과제
- `CGameDeck` 클래스는 `CDeck`과 `ToHTML` 인터페이스를 구현
  - `Channel`과 `MutableStateFlow`를 활용
- `CCPlayer` 클래스는 `CPlayer`와 `ToHTML` 인터페이스를 구현
  - 입력을 따로 받지 않고 알아서 어떤 카드를 버릴지 결정하는 점은 이전 과제에서의 computer player와 마찬가지
  - 하지만 무조건 손에 든 첫번째 카드를 버리는 것이 아니라 4개의 같은 숫자 카드를 모으기에 유리한 간단한 방식으로 put을 작성할 것 (그래야 돌리면 게임이 적당한 시간 안에 끝날 가능성이 커짐)
  - `Carditem`의 `up` 속성을 `true`로 하고 `toHTML`로 HTML 코드를 생성할 때도 카드 앞면이 드러나 보이도록 하라
- 덱(`CGameDeck`) 3개와 플레이어(`CCPlayer`) 3명으로 구성된 게임을 시뮬레이션하라
  - 처음 시작할 때는 하나의 덱에 52개의 전체 카드를 무작위 순서로 섞어서 모두 넣은 다음, 3명의 플레이어에게 각각 4장씩 draw 해주도록 게임을 초기화하라. 나머지 덱들은 물론 비어 있는 상태로 게임이 시작된다
  - 각각의 플레이어의 동작을 별도의 코루틴으로 동시다발적으로 동작하게 하라
  - 각 플레이어가 카드를 draw하거나 put할때마다 HTML 출력을 업데이트하라 (적절한 딜레이를 주어 눈으로 확인 가능한 정도의 속도로 진행되도록 하라)
  - 각각의 덱들도 일정 시간 간격으로 HTML 출력을 업데이트하라 (채널 안에 CardItem 정보가 들어있어서 직접 접근은 불가하지만 채널 안에 보관된 카드 개수를 잘 추적하여 카드 개수만큼 뒷면이 보이는 모양으로 겹쳐지게 지난 번 과제처럼 출력하라) 
  - 게임을 잘 종료할 수 있는 방법을 생각하여 작성하라

In [37]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*
import kotlinx.coroutines.flow.*

In [38]:
interface CDeck { // concurrent deck interface
    fun size(): Int
    suspend fun draw(): CardItem // 덱에서 카드를 뽑기
    suspend fun put(c: CardItem) // 덱에 카드 c를 놓기
}

interface CPlayer { // concurrent player interface
    val hand: MutableList<CardItem> // 이것도 가능!!
    suspend fun draw(d: Deck) // 덱 d로부터 카드 하나 뽑기
    suspend fun put(d: Deck)  // 카드 하나를 덱 d에 놓기
}

In [39]:
/*
class CGameDeck(/* */) : CDeck, ToHTML {
    // ...
    // ...
    override fun size() ...
    override suspend fun draw(): CardItem // 덱에서 카드를 뽑기
    override suspend fun put(c: CardItem) // 덱에 카드 c를 놓기
    override fun toHTML() ...
}
*/

In [40]:
/*
class CCPlayer(/* */) : CPlayer, ToHTML { // concurrent 
    // ...
    // override val hand ...
    override fun size() ...
    override suspend fun draw(): CardItem // 덱에서 카드를 뽑기
    override suspend fun put(c: CardItem) // 덱에 카드 c를 놓기
    override fun toHTML() ...
}
*/

In [41]:
DISPLAY( HTML("hello0").withId("player0") )
DISPLAY( HTML("hello1").withId("player1") )

z

Z

In [42]:
import kotlinx.coroutines.*
/*
val d0 = CGameDeck(..)
val d1 = CGameDeck(..)
val p0 = CCPlayer(..)
val p1 = CCPlayer(..)
*/
runBlocking {
    launch {
        for (x in 'a'..'z') {
            // p0.draw(d0)
            // delay(200)
            // UPDATE_DISPLAY( HTML(x.toString()), "player0")
            // p0.put(d1)
            delay(200)
            UPDATE_DISPLAY( HTML(x.toString()), "player0")
        }
    }

    launch {
        for (x in 'A'..'Z') {
            // p1.draw(d1)
            // delay(200)
            // UPDATE_DISPLAY( HTML(x.toString()), "player1")
            // p1.put(d0)
            delay(150)
            UPDATE_DISPLAY( HTML(x.toString()), "player1")
        }
    }
    
    // 메인 작업이 바로 게임 마스터
    delay(5000)
    println("Done!")
}

Done!
